In [6]:
import boto3


region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

athena_client = boto_session.client(service_name="athena", region_name=region)

default_s3_bucket_name = "s3://sagemaker-us-west-1-798223350085/athena/"


[01/24/25 22:03:48] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=204989;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=183169;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

In [3]:
query = "SELECT id, sum(payment) from feature01 group by id;"

query_response = athena_client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={"Database": "customer"},
    ResultConfiguration={
        "OutputLocation": default_s3_bucket_name,
    },
)

print(query)

SELECT id, sum(payment) from feature01 group by id;


In [4]:
import pandas as pd

S3_BUCKET_NAME = "sagemaker-us-west-1-798223350085"
S3_OUTPUT_DIRECTORY = "athena"
temp_file_location: str = "./athena_query_results.csv"
s3_client = boto3.client(
    service_name="s3",
    region_name=region,
)

s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
    temp_file_location,
)
df = pd.read_csv(temp_file_location)

In [22]:
import sagemaker
from sagemaker.session import Session


region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)
region

[01/25/25 11:26:21] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=817161;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=630399;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

'us-west-1'

In [23]:
# read dynamodb feature
dynamodb = boto3.client('dynamodb')
record = dynamodb.get_item(TableName='customer', Key={'id':{'S':'1'}})

In [10]:
children=record['Item']['children']['N']
children

'2'

In [12]:
# Offline feature from athena
import pandas as pd
import json
churn = pd.read_csv("athena_query_results.csv")

In [13]:
id = churn['id'].values[0]
print(id)

1


In [14]:
payments = churn["_col1"].values[0]
payments

20

In [25]:
from sagemaker.session import Session
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
boto_session = boto3.Session(region_name=region)

In [36]:
# write fo feature store
from sagemaker.feature_store.feature_group import FeatureGroup
user_feature_group = FeatureGroup(
    name="customer", sagemaker_session=sagemaker_session
)
data = [[str(id), int(children), payments, '2023-07-01T14:59:55.711Z']]

df = pd.DataFrame(data, columns=['id', 'children', 'payment', 'event_time'])
df

,id,children,payment,event_time
0,101,100,100,2023-07-01T14:59:55.711Z


In [48]:
user_feature_group.ingest(data_frame=df, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='customer', feature_definitions={'id': {'FeatureName': 'id', 'FeatureType': 'String'}, 'children': {'FeatureName': 'children', 'FeatureType': 'Integral'}, 'event_time': {'FeatureName': 'event_time', 'FeatureType': 'String'}, 'payment': {'FeatureName': 'payment', 'FeatureType': 'Integral'}}, sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7f9153ce8700>, sagemaker_session=<sagemaker.session.Session object at 0x7f9153ceb370>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7f9154057550>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [50]:
# read from feature store
# record_identifier_value is the primary key in feature store, e.g. id in our case
record_identifier_value = str(101)

record = featurestore_runtime.get_record(
    FeatureGroupName="customer",
    RecordIdentifierValueAsString=record_identifier_value,
)
record

{'ResponseMetadata': {'RequestId': '8ae8ec9a-3b42-4da6-8a9d-623aa02a70ba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8ae8ec9a-3b42-4da6-8a9d-623aa02a70ba',
   'content-type': 'application/json',
   'content-length': '341',
   'date': 'Sat, 25 Jan 2025 19:47:53 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'id', 'ValueAsString': '101'},
  {'FeatureName': 'children', 'ValueAsString': '100'},
  {'FeatureName': 'event_time', 'ValueAsString': '2023-07-01T14:59:55.711Z'},
  {'FeatureName': 'payment', 'ValueAsString': '100'}]}